In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
q = """
with seat_ids as
    (select distinct seat_id, section_name, row_name, seat, seat_x, seat_y
    from custom.cth_xy_coordinates),
historical_data as
    (select cth_historical_all_1718_2223.season, section_name || '-' || row_name || '-' || seat as seat_id,
        gross_revenue, ticket_type, paid_seats, comp_seats, paid_attended+comp_attended as attended
    from custom.cth_historical_all_1718_2223
    left join custom.cth_game_descriptions on date(cth_historical_all_1718_2223.event_date) = date(cth_game_descriptions.event_date)
    where game_type = 1),
current_data as
    (select '2023-24' as season, cth_v_ticket_2324.seat_id, gross_revenue, cth_v_ticket_2324.ticket_type,
       cth_v_ticket_2324.paid_seats, cth_v_ticket_2324.comp_seats,
       case
           when attendance_scan_datetime > 1 then 1
            else 0
        end as attended
    from custom.cth_v_ticket_2324
    left join custom.cth_v_attendance_2324 on cth_v_ticket_2324.seat_event_id = cth_v_attendance_2324.seat_event_id),
all_data as
    (select *
    from historical_data
    union all
    select *
    from current_data),
all_data_agg as
    (select season, seat_id, sum(gross_revenue) as gross_revenue, sum(paid_seats) as times_paid,
        sum(comp_seats) as times_comp, sum(attended) as attended, sum(CASE WHEN ticket_type = 'Full' THEN 1 ELSE 0 END) AS times_full,
        sum(CASE WHEN ticket_type = 'Premier' THEN 1 ELSE 0 END) AS times_premier,
        sum(CASE WHEN ticket_type in ('Sponsor', 'Trade') THEN 1 ELSE 0 END) AS times_other,
        sum(CASE WHEN ticket_type in ('Flex', 'Groups', 'Nightly Suites', 'Secondary', 'Singles') THEN 1 ELSE 0 END) AS times_nightly
     from all_data
     group by season, seat_id)
select season, seat_ids.seat_id, section_name, row_name, seat, seat_x, seat_y, gross_revenue, times_paid, times_comp,
       times_full, times_premier, times_other, times_nightly, attended
from seat_ids
left join all_data_agg on seat_ids.seat_id = all_data_agg.seat_id
"""
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

,season,seat_id,section_name,row_name,seat,seat_x,seat_y,gross_revenue,times_paid,times_comp,times_full,times_premier,times_other,times_nightly
0,2017-18,101-1-1,101,1,1,2524,2039,194.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2017-18,101-1-2,101,1,2,2508,2039,194.0,1.0,0.0,1.0,0.0,0.0,0.0
2,None,101-1-3,101,1,3,2492,2039,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,101-1-4,101,1,4,2476,2039,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-18,101-10-1,101,10,1,2604,1823,7704.0,35.0,5.0,15.0,0.0,0.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120534,2023-24,SL50-SRO-5,SL50,SRO,5,3509,3690,6520.6,37.0,2.0,0.0,0.0,0.0,2.0
120535,2017-18,SL50-SRO-6,SL50,SRO,6,3524,3686,155.0,1.0,3.0,0.0,0.0,0.0,4.0
120536,2023-24,SL50-SRO-6,SL50,SRO,6,3524,3686,0.0,0.0,1.0,0.0,0.0,0.0,1.0
120537,2017-18,SL50-SRO-7,SL50,SRO,7,3540,3681,155.0,1.0,3.0,0.0,0.0,0.0,4.0


In [ ]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "cth_arenamap_1718_2324")